In [ ]:
pip install -qU langchain_community tiktoken langchain-openai langchainhub chromadb langchain langgraph json dotenv

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['TAVILY_API_KEY']= os.getenv('TAVILY_API_KEY')

In [2]:
import json

file_path = 'data.json'  

with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

In [9]:
def extract_spending_details_by_category(data):
    categories = ['본식', '스드메', '예복', '예물/예단', '신혼여행', '가전/가구', '기타']
    
    result = {category: [] for category in categories}
    
    for category_data in data['groupedSpendings']:
        category_name = category_data['budgetCategory']['budgetCategoryName']
        # totalAmount은 사용 x
        #ipdb.set_trace()
        
        if category_name in categories:
            for spending in category_data['spendings']:
                spending_details = {
                    f"{spending['title']}": spending['amount']
                }
                result[category_name].append(spending_details)
    
    return result


extracted_budget_data = extract_spending_details_by_category(data)

print(extracted_budget_data)

{'본식': [{'식대': 0}, {'홀 대관료': 1000000}], '스드메': [{'스드메': 1439000}], '예복': [{'예복': 0}], '예물/예단': [{'결혼반지': 2390000}], '신혼여행': [{'숙박비': 0}, {'항공권': 0}, {'투어 및 기타 경비': 0}], '가전/가구': [{'청소기': 759000}, {'침대': 8102000}, {'삼성': 8770000}, {'식탁': 722000}, {'쇼파': 3204820}], '기타': [{'줄눈': 450000}, {'탄성코트': 550000}, {'하자점검': 270000}, {'입주청소': 510000}, {'TV(무타공)': 330000}]}


In [7]:
# English 
PROMPT_TEMPLATE_EG = """
# Instruction 
You are an AI assistant helping with wedding budget management. Please respond to the user's questions kindly and accurately. Your goal is to provide helpful advice based on the user’s current wedding budget, suggest possible savings or cost-efficient alternatives, and, when needed, provide additional information by searching the internet for price comparisons, recommendations, or tips.

Current wedding budget information:
{budget_info}


# GUIDELINES:

1. Always consider the user’s wedding budget limitations and provide cost-efficient suggestions where possible.
2. When there are zero values or missing information in certain categories, offer advice on possible cost ranges or alternatives based on typical wedding expenditures.
3. For large expenses, help the user identify areas where they might save by comparing current prices online or recommending alternative options (e.g., for furniture or honeymoon).
4. If there are optional categories, ask the user if they would like to explore further options for those categories (e.g., entertainment, videography).
5. Only respond to queries that are directly related to wedding budget management or planning.
6. Keep your tone friendly, empathetic, and professional.

# IMPORTANT : Answer in KOREAN

# Begin!

Previous conversation history:
{chat_history}

New input: {input}

"""

In [10]:
import json

from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate


class BudgetAssistant:
    def __init__(self, budget_data):
        """
        예산 어시스턴트 초기화
        
        Args:
            budget_data (dict): 사용자의 예산 정보
        """
        
        self.budget_data = budget_data
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            input_key="input"
        )
        
        self.prompt = PromptTemplate(
            input_variables=["chat_history", "input", "budget_info"],
            template=PROMPT_TEMPLATE_EG
        )
        
        self.llm = ChatOpenAI(
            model="gpt-4o-mini",
            temperature=0.1
        )
        
        self.chain = LLMChain(
            llm=self.llm,
            prompt=self.prompt,
            memory=self.memory
        )
    
    def ask(self, question):
        """
        질문에 대한 응답 생성
        
        Args:
            question (str): 사용자 질문
        
        Returns:
            str: LLM의 응답
        """
        return self.chain.run(
            input=question,
            budget_info=json.dumps(self.budget_data, ensure_ascii=False, indent=2)
        )

def main():
    
    # 예산 데이터
    extracted_budget_data = extract_spending_details_by_category(data)

    # 예산 어시스턴트 초기화
    assistant = BudgetAssistant(extracted_budget_data)
    
    print("예산 관리 어시스턴트입니다. 질문해 주세요! (종료하려면 'quit' 입력)")
    
    while True:
        user_input = input("\n질문: ")
        
        if user_input.lower() == 'quit':
            print("상담을 종료합니다. 감사합니다!")
            break
            
        try:
            response = assistant.ask(user_input)
            print("\n답변:", response)
        except Exception as e:
            print(f"\n오류가 발생했습니다: {str(e)}")

if __name__ == "__main__":
    main()

예산 관리 어시스턴트입니다. 질문해 주세요! (종료하려면 'quit' 입력)

답변: 안녕하세요! 결혼 준비에 대해 궁금한 점이 있으신가요? 예산 관리나 비용 절감 방법에 대해 도와드릴 수 있습니다. 어떤 부분에 대해 이야기해볼까요?

답변: 물론입니다! 현재 제공해주신 예산 정보를 바탕으로 분석해드리겠습니다.

### 현재 예산 요약:
1. **본식**
   - 홀 대관료: 1,000,000원
   - 식대: 0원 (식대는 보통 1인당 5만 원에서 10만 원 정도 예상할 수 있습니다. 손님 수에 따라 예산을 고려해보세요.)

2. **스드메 (스튜디오, 드레스, 메이크업)**
   - 총 비용: 1,439,000원

3. **예복**
   - 예복: 0원 (예복 비용도 고려해보셔야 합니다. 대여나 구매를 통해 50만 원에서 200만 원 정도 예상할 수 있습니다.)

4. **예물/예단**
   - 결혼반지: 2,390,000원

5. **신혼여행**
   - 숙박비, 항공권, 투어 및 기타 경비: 0원 (신혼여행 예산을 설정해보세요. 평균적으로 100만 원에서 300만 원 정도가 필요할 수 있습니다.)

6. **가전/가구**
   - 총 비용: 8,790,820원 (가전 및 가구는 큰 비용이 발생하므로, 필요한 품목을 우선순위에 따라 조정해보세요.)

7. **기타**
   - 총 비용: 2,110,000원

### 총 예산
현재까지의 총 비용은 약 15,429,820원입니다. 여기에 추가적으로 식대, 예복, 신혼여행 비용을 고려해야 합니다.

### 비용 절감 제안:
- **식대**: 손님 수를 줄이거나, 뷔페식으로 변경하면 비용을 절감할 수 있습니다.
- **예복**: 대여를 고려하거나, 세일 시즌에 구매하면 비용을 줄일 수 있습니다.
- **신혼여행**: 가까운 국내 여행지나 패키지 상품을 이용하면 비용을 절감할 수 있습니다.
- **가전/가구**: 중고 제품이나 세일을 통해 구매하면 큰 비용을 아낄 수 있습니다.

추가적으로 궁금한 부분이나 더 알고